# Exploración, Transformación y Limpieza

In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
pd.set_option('display.max_columns', None)

## Análisis del fichero *finanzas-hotel-bookings.csv*.

In [2]:
# Función para el análisis inicial de los datos facilitados

def analisis_inicial(file_path):
    # Importamos el fichero origen formato .csv
    df = pd.read_csv(file_path, index_col=0)

    # Exploramos el título de las columnas
    print("Título de las columnas:")
    print(df.columns)

    # Exploramos el número de duplicados en el DataFrame
    num_duplicates = df.duplicated().sum()
    print(f"\nNúmero de duplicados en el DataFrame: {num_duplicates}")

    # Mostramos las primeras filas del DataFrame
    print(df.info())

    # Devolvemos el DataFrame creado
    return df

In [3]:
df_raw_data = analisis_inicial("finanzas-hotel-bookings.csv")

C:\Users\rocio\AppData\Local\Temp\ipykernel_12208\4063040639.py:5: DtypeWarning: Columns (1,2,5,13,14,15,16,20,21,26,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, index_col=0)


Título de las columnas:
Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'agent', 'company',
       'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', '0'],
      dtype='object')

Número de duplicados en el DataFrame: 63040
<class 'pandas.core.frame.DataFrame'>
Index: 182877 entries, 0 to 182876
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   --

## Limpieza preliminar

In [4]:
def limpieza_preliminar(df):
    # Eliminamos duplicados
    df_sin_duplicados = df.drop_duplicates()
    
    # Filtramos las filas donde '0' sea nulo y creamos nuevo df
    df_sin_0 = df_sin_duplicados[df_sin_duplicados['0'].isnull()]
    
    # Eliminamos la columna '0' por el alto número de nulos
    df_sin_0 = df_sin_0.drop(columns=['0'])
    
    # Filtramos las filas donde 'hotel' no sea nulo para eliminar las filas que no añaden información
    df = df_sin_0[df_sin_0['hotel'].notnull()]
    
    # Reseteamos el índice del DataFrame resultante
    df = df.reset_index(drop=True)
    
    return df

In [5]:
df = limpieza_preliminar(df_raw_data)

# Función para explorar y limpiar las columnas según su tipo de dato

In [6]:
def info_col(df, dtype=None):
    if dtype == 'number':
        columns = df.select_dtypes(include=['Int64', 'int64', 'float64']).columns
    elif dtype == 'object':
        columns = df.select_dtypes(include=['object']).columns
    elif dtype == 'bool':
        columns = df.select_dtypes(include=['bool']).columns
    elif dtype == 'datetime':
        columns = df.select_dtypes(include=['datetime64[ns]'])
    else:
        return "Tipo de dato no compatible."

    # Calculamos el porcentaje de valores nulos en df para todas las columnas
    df_nulos = pd.DataFrame((df.isnull().sum() / df.shape[0]) * 100, columns=["%_nulos"])

    # Filtramos por tipo de dato
    if dtype:
        filtered_columns = df.select_dtypes(include=dtype).columns
        df = df[filtered_columns]
        df_nulos = pd.DataFrame((df.isnull().sum() / df.shape[0]) * 100, columns=["%_nulos"])

    # Obtenemos información para cada columna
    info_list = []
    for column in df.columns:
        unique_values = df[column].unique()
        dtype = df[column].dtype

        if dtype == 'object':
            unique_values_sorted = sorted(unique_values, key=str)
            extra_info = {
                'top': df[column].mode().values[0],
                'freq': df[column].value_counts().iloc[0]
            }
        elif dtype in ['Int64', 'int64', 'float64']:
            unique_values_sorted = df[column].dropna().unique()
            describe_stats = df[column].describe()
            extra_info = {
                'mean': describe_stats['mean'],
                'mode': df[column].mode().values[0],
                'min': describe_stats['min'],
                '25%': describe_stats['25%'],
                '50%': describe_stats['50%'],
                '75%': describe_stats['75%'],
                'max': describe_stats['max'],
                'std': describe_stats['std']
            }
        elif dtype == 'bool':
            unique_values_sorted = unique_values
            extra_info = {
                'top': df[column].mode().values[0],
                'freq': df[column].value_counts().iloc[0]
            }
        elif dtype in ['datetime64[ns]']:
            unique_values_sorted = sorted(unique_values)
            describe_stats = df[column].describe()
            extra_info = {
                'mean': describe_stats['mean'],
                'mode': df[column].mode().values[0],
                'min': describe_stats['min'],
                '25%': describe_stats['25%'],
                '50%': describe_stats['50%'],
                '75%': describe_stats['75%'],
                'max': describe_stats['max']
            }
        else:
            unique_values_sorted = unique_values
            extra_info = {}

        info = {
            'count': df[column].count(),
            '#unique': df[column].nunique(),
            'unique': unique_values_sorted,
            'dtype': dtype,
            **extra_info
        }
        info_list.append(info)

    # Creamos un DataFrame con la información recopilada
    df_info = pd.DataFrame(info_list, index=df.columns)

    # Unir el DataFrame de porcentaje de nulos con la información obtenida
    result = pd.concat([df_nulos, df_info], axis=1)

    # Ordenar por %_nulos de mayor a menor
    result = result.sort_values(by="%_nulos", ascending=False)

    # Redondear el valor de %_nulos a 2 decimales
    result['%_nulos'] = result['%_nulos'].round(2)

    return result

## Limpieza columnas object

In [7]:
info_object_inic = info_col(df, dtype='object')
info_object_inic

,%_nulos,count,#unique,unique,dtype,top,freq
market_segment,49.80,59240,8,"[Aviation, Complementary, Corporate, Direct, G...",object,Online TA,28313
country,45.55,64251,163,"[ABW, AGO, AIA, ALB, AND, ARE, ARG, ARM, ASM, ...",object,PRT,25903
reserved_room_type,33.73,78198,10,"[A, B, C, D, E, F, G, H, L, P, nan]",object,A,56144
customer_type,21.26,92912,4,"[Contract, Group, Transient, Transient-Party, ...",object,Transient,69800
distribution_channel,11.41,104539,5,"[Corporate, Direct, GDS, TA/TO, Undefined, nan]",object,TA/TO,85534
reservation_status_date,10.90,105137,976,"[2014-10-17 00:00:00, 2014-11-18 00:00:00, 201...",object,2015-10-21 00:00:00,1232
hotel,0.00,118005,2,"[City Hotel, Resort Hotel]",object,City Hotel,78066
is_canceled,0.00,118005,2,"[False, True]",object,False,74883
arrival_date_month,0.00,118005,15,"[1, 2, 3, April, August, December, February, J...",object,August,13751
meal,0.00,118005,5,"[BB, FB, HB, SC, Undefined]",object,BB,91207


In [8]:
def limpieza_object(df):
    # Mapeamos valores en 'arrival_date_month'
    numeros_a_meses = {
        '1': 'January',
        '2': 'February',
        '3': 'March',
    }
    df['arrival_date_month'] = df['arrival_date_month'].map(numeros_a_meses).fillna(df['arrival_date_month'])

    # Reemplazamos valores específicos en 'country'
    df['country'] = df['country'].replace("CN", "CHN")

    # Completamos valores faltantes nan en columnas específicas
    columns_to_fill = ['market_segment', 'country', 'reserved_room_type', 'customer_type', 'distribution_channel']
    df[columns_to_fill] = df[columns_to_fill].fillna('Undefined')

    # Reemplazamos valores 'Undefined' en 'reserved_room_type' por el valor adyacente de 'assigned_room_type'
    mask = df['reserved_room_type'] == 'Undefined'
    df.loc[mask, 'reserved_room_type'] = df.loc[mask, 'assigned_room_type']

    # Convertimos 'reservation_status_date' a datetime, forzando la conversión de los errores (nan = NaT, otros formatos son 30 y valores entre 2022 y)
    df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'], errors='coerce')

    return df

In [9]:
df = limpieza_object(df)

In [10]:
info_object_final = info_col(df, "object").sort_index(axis=0)
info_object_final

,%_nulos,count,#unique,unique,dtype,top,freq
arrival_date_month,0.0,118005,12,"[April, August, December, February, January, J...",object,August,13751
assigned_room_type,0.0,118005,12,"[A, B, C, D, E, F, G, H, I, K, L, P]",object,A,72716
country,0.0,118005,163,"[ABW, AGO, AIA, ALB, AND, ARE, ARG, ARM, ASM, ...",object,Undefined,53754
customer_type,0.0,118005,5,"[Contract, Group, Transient, Transient-Party, ...",object,Transient,69800
distribution_channel,0.0,118005,5,"[Corporate, Direct, GDS, TA/TO, Undefined]",object,TA/TO,85534
hotel,0.0,118005,2,"[City Hotel, Resort Hotel]",object,City Hotel,78066
is_canceled,0.0,118005,2,"[False, True]",object,False,74883
market_segment,0.0,118005,8,"[Aviation, Complementary, Corporate, Direct, G...",object,Undefined,58766
meal,0.0,118005,5,"[BB, FB, HB, SC, Undefined]",object,BB,91207
reservation_status,0.0,118005,3,"[Canceled, Check-Out, No-Show]",object,Check-Out,74883


## Limpieza columnas bool

In [11]:
def limpieza_bool(df, columnas):
    for columna in columnas:
        if columna in df.columns:
            if set(df[columna].dropna().unique()) == {1, 0}:
                # Reemplazar valores en la columna
                df[columna].replace({1: True, 0: False}, inplace=True)
            # Convertir la columna al tipo de datos booleano
            df[columna] = df[columna].astype(bool)
    return df

In [12]:
df = limpieza_bool(df, ['is_repeated_guest', 'is_canceled'])

In [13]:
info_bool = info_col(df, dtype='bool')
info_bool

,%_nulos,count,#unique,unique,dtype,top,freq
is_canceled,0.0,118005,2,"[False, True]",bool,False,74883
is_repeated_guest,0.0,118005,2,"[False, True]",bool,False,109363


In [14]:
# Reemplazar valores en la columna 'is_repeated_guest'
#df['is_repeated_guest'].replace({1: True, 0: False}, inplace=True)
# Convertir la columna al tipo de datos booleano
#df['is_repeated_guest'] = df['is_repeated_guest'].astype('bool')

# Convertir 'is_canceled' a tipo booleano
#df['is_canceled'] = df['is_canceled'].astype(bool)

## Limpieza columnas Fecha: 
mantenemos nulos (vacíos)

In [15]:
# Análisis inicial de valores atípicos (antes de la transformación), nos dice que sólo hay 30 valores que no tienen formato datetyme, y que son años entre 2022 y 2030
#filtered_df = df[df['reservation_status_date'].notnull() & df['reservation_status_date'].astype(str).str.contains(r'\d{4}-\d{2}-\d{2}$')]
#filtered_df['reservation_status_date'].unique()
#filtered_df['reservation_status_date'].describe()

In [16]:
info_date = info_col(df, dtype="datetime")
info_date

,%_nulos,count,#unique,unique,dtype,mean,mode,min,25%,50%,75%,max
reservation_status_date,10.93,105107,946,"[2014-10-17 00:00:00, 2014-11-18 00:00:00, 201...",datetime64[ns],2016-07-30 20:45:16.637331456,2015-10-21,2014-10-17,2016-02-01,2016-07-27,2017-02-09,2017-12-08


## Limpieza columnas numéricas

In [17]:
info_num_inic = info_col(df, dtype='number')
info_num_inic

,%_nulos,count,#unique,unique,dtype,mean,mode,min,25%,50%,75%,max,std
company,96.74,3849,310,"[110.0, 270.0, 240.0, 154.0, 144.0, 307.0, 268...",float64,188.885685,40.0,6.00,62.00,178.0,270.0,543.0,131.449456
arrival_date_year,45.75,64023,3,"[2015.0, 2016.0, 2017.0]",float64,2016.158912,2016.0,2015.00,2016.00,2016.0,2017.0,2017.0,0.706380
children,42.05,68381,14,"[0.0, 1.0, 2.0, 10.0, 19.0, 3.0, 17.0, 11.0, 1...",float64,0.119887,0.0,0.00,0.00,0.0,0.0,19.0,0.611137
previous_cancellations,36.32,75146,21,"[0.0, 14.0, 10.0, 19.0, 13.0, 17.0, 1.0, 2.0, ...",float64,0.099260,0.0,0.00,0.00,0.0,0.0,26.0,0.959910
arrival_date_week_number,15.54,99667,53,"[27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34....",float64,27.140969,33.0,1.00,16.00,28.0,38.0,53.0,13.620939
agent,13.69,101855,333,"[304.0, 240.0, 303.0, 15.0, 241.0, 8.0, 250.0,...",float64,87.074881,9.0,1.00,9.00,14.0,229.0,535.0,110.988612
arrival_date_day_of_month,0.10,117886,31,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...",float64,15.803149,17.0,1.00,8.00,16.0,23.0,31.0,8.782959
booking_changes,0.00,118005,22,"[3.0, 4.0, 0.0, 1.0, 2.0, 5.0, 17.0, 18.0, 6.0...",float64,0.231956,0.0,0.00,0.00,0.0,0.0,21.0,0.744873
required_car_parking_spaces,0.00,118005,5,"[0.0, 1.0, 2.0, 8.0, 3.0]",float64,0.063252,0.0,0.00,0.00,0.0,0.0,8.0,0.246632
adr,0.00,118005,8879,"[0.0, 75.0, 98.0, 107.0, 103.0, 82.0, 105.5, 1...",float64,101.916935,62.0,-6.38,69.12,94.5,126.0,5400.0,50.723217


In [42]:
# eliminar la columna company por el alto % de nulos (96.74%)
#df = df.drop(columns=['company'])

In [43]:
# Reemplazar valores negativos en la columna 'adr' por 0
#df.loc[df['adr'] < 0.0, 'adr'] = 0.0

In [44]:
# Obtener las columnas float64
#float_columns = df.select_dtypes(include=['float64']).columns
#filtered_float_columns = float_columns[~float_columns.isin(['adr'])]

# Reemplazar NaN por 9999 en las columnas float64
#df[float_columns] = df[float_columns].fillna(9999.0)

# Convertir las columnas a tipo Int64
#for col in filtered_float_columns:
    #if (df[col] % 1 == 0).all():  # Comprobar si todos los valores son enteros
        #df[col] = df[col].astype('Int64')

In [22]:
def limpieza_num(df):
    # Eliminar la columna 'company' por alto % de nulos
    df = df.drop(columns=['company'])

    # Reemplazar valores negativos en la columna 'adr' por 0
    df.loc[df['adr'] < 0.0, 'adr'] = 0.0

    # Obtener las columnas float64 excepto 'adr'
    float_columns = df.select_dtypes(include=['float64']).columns
    filtered_float_columns = float_columns[~float_columns.isin(['adr'])]

    # Reemplazar NaN por 9999 en las columnas float64 excepto 'adr'
    df[filtered_float_columns] = df[filtered_float_columns].fillna(9999.0)

    # Convertir las columnas a tipo Int64 si todos los valores son enteros
    for col in filtered_float_columns:
        if (df[col] % 1 == 0).all():
            df[col] = df[col].astype('Int64')

    return df

In [23]:
df = limpieza_num(df)

In [25]:
info_num_fin = info_col(df, dtype='number')
info_num_fin

,%_nulos,count,#unique,unique,dtype,mean,mode,min,25%,50%,75%,max,std
lead_time,0.0,118005,479,"[342, 737, 7, 13, 14, 0, 9, 85, 75, 23, 35, 68...",Int64,102.962247,0.0,0.0,18.00,68.0,159.0,737.0,106.227209
previous_cancellations,0.0,118005,22,"[9999, 0, 14, 10, 19, 13, 17, 1, 2, 3, 26, 25,...",Int64,3631.664760,0.0,0.0,0.00,0.0,9999.0,9999.0,4808.699567
required_car_parking_spaces,0.0,118005,5,"[0, 1, 2, 8, 3]",Int64,0.063252,0.0,0.0,0.00,0.0,0.0,8.0,0.246632
adr,0.0,118005,8878,"[0.0, 75.0, 98.0, 107.0, 103.0, 82.0, 105.5, 1...",float64,101.916989,62.0,0.0,69.12,94.5,126.0,5400.0,50.723104
days_in_waiting_list,0.0,118005,128,"[0, 50, 47, 65, 122, 75, 101, 150, 125, 14, 60...",Int64,2.267650,0.0,0.0,0.00,0.0,0.0,391.0,17.410199
agent,0.0,118005,334,"[9999, 304, 240, 303, 15, 241, 8, 250, 115, 5,...",Int64,1443.607152,9.0,1.0,9.00,28.0,240.0,9999.0,3408.283646
booking_changes,0.0,118005,22,"[3, 4, 0, 1, 2, 5, 17, 18, 6, 14, 15, 10, 11, ...",Int64,0.231956,0.0,0.0,0.00,0.0,0.0,21.0,0.744873
previous_bookings_not_canceled,0.0,118005,73,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",Int64,0.138706,0.0,0.0,0.00,0.0,0.0,72.0,1.506125
babies,0.0,118005,14,"[0, 1, 18, 2, 17, 13, 15, 16, 12, 10, 11, 19, ...",Int64,0.015152,0.0,0.0,0.00,0.0,0.0,19.0,0.339671
arrival_date_year,0.0,118005,4,"[2015, 9999, 2016, 2017]",Int64,5667.951019,9999.0,2015.0,2016.00,2017.0,9999.0,9999.0,3976.961753


In [26]:
df.to_csv('FHB_limpio_1.csv', index=True)

## Filtramos por is_canceled == True

In [28]:
df_cancelled = df[df['is_canceled'] == True].copy()

In [29]:
df_cancelled

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
8,Resort Hotel,True,85,2015,July,27,1,0,3,2,0,0,BB,PRT,Undefined,Undefined,False,9999,0,A,A,0,240,0,Transient,82.0,0,1,Canceled,2015-05-06
9,Resort Hotel,True,75,2015,July,27,1,0,3,2,0,0,HB,Undefined,Undefined,TA/TO,False,0,0,D,D,0,15,0,Transient,105.5,0,0,Canceled,2015-04-22
10,Resort Hotel,True,23,9999,July,27,1,0,4,2,0,0,BB,PRT,Undefined,TA/TO,False,0,0,E,E,0,240,0,Undefined,123.0,0,0,Canceled,2015-06-23
27,Resort Hotel,True,60,9999,July,27,1,2,5,2,9999,0,BB,PRT,Undefined,TA/TO,False,0,0,E,E,0,240,0,Undefined,107.0,0,2,Canceled,2015-05-11
32,Resort Hotel,True,96,9999,July,9999,1,2,8,2,9999,0,BB,PRT,Direct,Direct,False,0,0,E,E,0,9999,0,Transient,108.3,0,2,Canceled,2015-05-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108938,City Hotel,True,132,9999,April,17,25,0,0,0,0,0,BB,FRA,Online TA,TA/TO,False,0,0,D,K,1,9,0,Transient,0.0,0,3,Canceled,NaT
110013,City Hotel,True,4,2017,June,23,5,1,0,1,9999,0,BB,Undefined,Undefined,Corporate,True,0,4,A,A,0,9999,0,Transient,65.0,0,0,Canceled,2017-05-06
110581,City Hotel,True,7,9999,May,22,31,0,1,57,0,0,BB,PRT,Undefined,Corporate,True,9999,1,A,A,0,9999,0,Undefined,65.0,0,0,Canceled,2017-05-31
110582,City Hotel,True,6,2017,July,29,17,1,0,52,0,0,BB,Undefined,Corporate,Corporate,True,1,1,A,D,0,9999,0,Transient,65.0,0,0,No-Show,2017-07-17


In [30]:
df_cancelled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43122 entries, 8 to 115927
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   hotel                           43122 non-null  object        
 1   is_canceled                     43122 non-null  bool          
 2   lead_time                       43122 non-null  Int64         
 3   arrival_date_year               43122 non-null  Int64         
 4   arrival_date_month              43122 non-null  object        
 5   arrival_date_week_number        43122 non-null  Int64         
 6   arrival_date_day_of_month       43122 non-null  Int64         
 7   stays_in_weekend_nights         43122 non-null  Int64         
 8   stays_in_week_nights            43122 non-null  Int64         
 9   adults                          43122 non-null  Int64         
 10  children                        43122 non-null  Int64         
 11  babies

In [31]:
df_cancelled.to_csv('cancelaciones_hotel.csv', index=True)